# Med-Data FL or no FL?

-> No FL

In [12]:
# import
import dask.dataframe as dd
from sklearn.preprocessing import MinMaxScaler
import os

## Ingest

In [11]:
df_path = "../original_work/med-insurance/data/insurance.csv"
ddf = dd.read_csv(df_path)
#ddf = ddf.repartition(npartitions = 1) # if it is small enough
ddf

,age,sex,bmi,children,smoker,region,charges
npartitions=1,,,,,,,
,int64,object,float64,int64,object,object,float64
,...,...,...,...,...,...,...


In [9]:
ddf.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Preprocess

In [87]:
from dask_ml.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler 
enc_ord = OrdinalEncoder()
enc_oh = OneHotEncoder()
scaler = MinMaxScaler()

ddf[['sex', 'smoker']] = enc_ord.fit_transform(
    ddf[['sex', 'smoker']].astype('category').categorize())

ddf[['region0', 'region1', 'region2', 'region3']] = enc_oh.fit_transform(
        ddf[["region"]].astype("category").categorize())

ddf[['age', 'bmi', 'children']] = scaler.fit_transform(ddf[['age', 'bmi', 'children']])

ddf.loc[:,ddf.columns != 'region'].head()

,age,sex,bmi,children,smoker,charges,region_0,region_1,region_2,region_3,region0,region1,region2,region3
0,0.021739,0,0.321227,0.0,0,16884.92400,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.000000,1,0.479150,0.2,1,1725.55230,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.217391,1,0.458434,0.6,1,4449.46200,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.326087,1,0.181464,0.0,1,21984.47061,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.304348,1,0.347592,0.0,1,3866.85520,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [79]:
ddf.compute()

,age,sex,bmi,children,smoker,region,charges,region_0,region_1,region_2,region_3
0,0.021739,0,0.321227,0.0,0,0,16884.92400,1.0,0.0,0.0,0.0
1,0.000000,1,0.479150,0.2,1,1,1725.55230,0.0,1.0,0.0,0.0
2,0.217391,1,0.458434,0.6,1,1,4449.46200,0.0,1.0,0.0,0.0
3,0.326087,1,0.181464,0.0,1,2,21984.47061,0.0,0.0,1.0,0.0
4,0.304348,1,0.347592,0.0,1,2,3866.85520,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1333,0.695652,1,0.403820,0.6,1,2,10600.54830,0.0,0.0,1.0,0.0
1334,0.000000,0,0.429379,0.0,1,3,2205.98080,0.0,0.0,0.0,1.0
1335,0.000000,0,0.562012,0.0,1,1,1629.83350,0.0,1.0,0.0,0.0
1336,0.065217,0,0.264730,0.0,1,0,2007.94500,1.0,0.0,0.0,0.0


In [81]:
# Functions for easy data handling and feature definition
training_features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region0', 'region1', 'region2', 'region3']
target_variable = 'charges'
def get_x(d):
    """Return the before defined training_features from the given data.

    :param d: The data to filter the features from
    :type d: pandas.DataFrame
    :return: The data of the training_features
    :rtype: pandas.DataFrame
    """
    return d[training_features]

def get_y(d):
    """Return the before defined target_variable from the given data.

    :param d: The data to filter the features from
    :type d: pandas.DataFrame
    :return: The data of the training_features
    :rtype: pandas.DataFrame
    """
    return d[[target_variable]]

In [88]:
from dask_ml.model_selection import train_test_split
# Divide data into train and test data
X_train, X_test, y_train, y_test = train_test_split(
    ddf[training_features], ddf[[target_variable]], 
    test_size=0.2, random_state=42, shuffle = True)
    

In [90]:
X_train.compute()

,age,sex,bmi,children,smoker,region0,region1,region2,region3
693,0.130435,1,0.207022,0.0,1,0.0,0.0,1.0,0.0
1322,0.956522,1,0.615281,0.0,1,0.0,1.0,0.0,0.0
481,0.673913,1,0.579769,0.4,1,0.0,1.0,0.0,0.0
1171,0.543478,0,0.288943,0.4,0,1.0,0.0,0.0,0.0
1259,0.739130,0,0.194243,0.0,1,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
653,0.586957,0,0.547215,0.4,1,0.0,1.0,0.0,0.0
535,0.434783,1,0.324590,0.2,1,0.0,0.0,0.0,1.0
1099,0.152174,0,0.485069,0.2,1,0.0,1.0,0.0,0.0
532,0.891304,1,0.369653,0.4,1,0.0,1.0,0.0,0.0


## Model

In [ ]:
# ...
# https://ml.dask.org/keras.html

## Conclusion

---

In [78]:
ddf.visualize()

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.637676 to fit
